# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import joblib
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
import string
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gletsong\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gletsong\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gletsong\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///disaster-response.db')
df = pd.read_sql_table('messages', engine)

In [3]:
X = df['message']
y = df.drop(['message', 'genre', 'id', 'original'], axis=1)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(word).lower().strip() for word in tokens]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
                    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                    ('clf', MultiOutputClassifier(RandomForestClassifier()))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [8]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x00000174FE733310>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(X_test)

In [10]:
y_pred

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int64)

In [11]:
y_pred.shape

(7857, 36)

In [12]:
for i, col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.72      0.27      0.40      1816
           1       0.81      0.97      0.88      5982
           2       0.47      0.15      0.23        59

    accuracy                           0.80      7857
   macro avg       0.67      0.46      0.50      7857
weighted avg       0.79      0.80      0.76      7857

request
              precision    recall  f1-score   support

           0       0.72      0.28      0.40      1816
           1       0.81      0.97      0.88      5982
           2       0.39      0.15      0.22        59

    accuracy                           0.80      7857
   macro avg       0.64      0.47      0.50      7857
weighted avg       0.79      0.80      0.77      7857

offer
              precision    recall  f1-score   support

           0       0.73      0.27      0.40      1816
           1       0.81      0.97      0.88      5982
           2       0.45      0.15      0.23        59

              precision    recall  f1-score   support

           0       0.73      0.27      0.39      1816
           1       0.81      0.97      0.88      5982
           2       0.37      0.12      0.18        59

    accuracy                           0.80      7857
   macro avg       0.63      0.45      0.48      7857
weighted avg       0.79      0.80      0.76      7857

fire
              precision    recall  f1-score   support

           0       0.72      0.27      0.39      1816
           1       0.81      0.97      0.88      5982
           2       0.32      0.12      0.17        59

    accuracy                           0.80      7857
   macro avg       0.62      0.45      0.48      7857
weighted avg       0.78      0.80      0.76      7857

earthquake
              precision    recall  f1-score   support

           0       0.73      0.27      0.39      1816
           1       0.81      0.97      0.88      5982
           2       0.50      0.15      0.23        59

    

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'tfidf', 'clf', 'tfidf__analyzer', 'tfidf__binary', 'tfidf__decode_error', 'tfidf__dtype', 'tfidf__encoding', 'tfidf__input', 'tfidf__lowercase', 'tfidf__max_df', 'tfidf__max_features', 'tfidf__min_df', 'tfidf__ngram_range', 'tfidf__norm', 'tfidf__preprocessor', 'tfidf__smooth_idf', 'tfidf__stop_words', 'tfidf__strip_accents', 'tfidf__sublinear_tf', 'tfidf__token_pattern', 'tfidf__tokenizer', 'tfidf__use_idf', 'tfidf__vocabulary', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_scor

In [14]:
parameters = {'clf__estimator__max_depth': [5, 15],
              'clf__estimator__min_samples_leaf': [2, 5],
              'clf__estimator__min_samples_split': [3, 7],
              'clf__estimator__n_estimators': [10, 25],
               'clf__estimator__max_features':[0.3,0.65]}

               
            

cv = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1)
cv.fit(X_train, y_train)

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x00000174FE733310>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=-1,
             param_grid={'clf__estimator__max_depth': [5, 15],
                         'clf__estimator__max_features': [0.3, 0.65],
                         'clf__estimator__min_samples_leaf': [2, 5],
                         'clf__estimator__min_samples_split': [3, 7],
                         'clf__estimator__n_estimators': [10, 25]})

In [15]:
cv.best_params_

{'clf__estimator__max_depth': 5,
 'clf__estimator__max_features': 0.3,
 'clf__estimator__min_samples_leaf': 5,
 'clf__estimator__min_samples_split': 3,
 'clf__estimator__n_estimators': 25}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
y_pred = cv.predict(X_test)

for i, col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))



related
              precision    recall  f1-score   support

           0       0.70      0.12      0.21      1816
           1       0.78      0.98      0.87      5982
           2       0.00      0.00      0.00        59

    accuracy                           0.78      7857
   macro avg       0.50      0.37      0.36      7857
weighted avg       0.76      0.78      0.71      7857

request
              precision    recall  f1-score   support

           0       0.70      0.11      0.20      1816
           1       0.78      0.99      0.87      5982
           2       0.00      0.00      0.00        59

    accuracy                           0.78      7857
   macro avg       0.49      0.37      0.36      7857
weighted avg       0.75      0.78      0.71      7857

offer
              precision    recall  f1-score   support

           0       0.76      0.08      0.14      1816
           1       0.77      0.99      0.87      5982
           2       0.00      0.00      0.00        59

C:\Users\gletsong\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.76      0.09      0.16      1816
           1       0.78      0.99      0.87      5982
           2       0.00      0.00      0.00        59

    accuracy                           0.78      7857
   macro avg       0.51      0.36      0.34      7857
weighted avg       0.77      0.78      0.70      7857

death
              precision    recall  f1-score   support

           0       0.72      0.12      0.20      1816
           1       0.78      0.99      0.87      5982
           2       0.00      0.00      0.00        59

    accuracy                           0.78      7857
   macro avg       0.50      0.37      0.36      7857
weighted avg       0.76      0.78      0.71      7857

other_aid
              precision    recall  f1-score   support

           0       0.69      0.12      0.20      1816
           1       0.78      0.98      0.87      5982
           2       0.00      0.00      0.00        59

    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [17]:
joblib.dump(cv.best_estimator_, 'disaster_model.pkl')

['disaster_model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.